<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/Erotic%E7%9A%84%E5%B0%8F%E8%AF%B4%E6%95%B0%E6%8D%AE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 统计换行数量
- [x] 统计引号数量
- [ ] prompt改写成多样化的prompt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Wuxia/erotica

In [3]:
!pip install -q tiktoken
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [17]:
#@title 定义strong_divide
def divide_str(s, sep=['\n', '.', '。'], consider_quote = True ):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len + mid_len - 2)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            if consider_quote == True:
                if s[sep_pos + 1] in ['"', "'","“","」"]:
                    sep_pos += 1

            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s, balance_ratio = 0.25):
    left_bar = int(len(s) * balance_ratio)
    left, right = divide_str(s, '\n')
    if len(left) >= left_bar and len(right) >= left_bar:
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)
    if len(left) >= left_bar and len(right) >= left_bar:
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

def super_divide( s, max_len_token = 300, iter_deep = 1):
    n_token = len(enc.encode(s))
    if n_token <= max_len_token or iter_deep > 2:
        return [s]

    left, right = strong_divide(s)
    return super_divide(left, max_len_token, iter_deep+1) + \
            super_divide(right, max_len_token, iter_deep+1)

对于 /content/drive/MyDrive/Wuxia/erotica 目录下的所有文件夹

如 /content/drive/MyDrive/Wuxia/erotica/例子小说1

这个时候我需要定义

source = "例子小说1"

对于这个目录下面，有很多txt

我希望对每个txt文件进行读取

保存到一个list of dict , datas中，保存source 和text两个字段

请用python为我实现

In [50]:
import os

# 定义目录路径
directory_path = "/content/drive/MyDrive/Wuxia/erotica"

# 初始化用于保存数据的列表
datas = []

from tqdm import tqdm

count = 0

import time

# 遍历目录下的所有文件夹
for folder in tqdm(os.listdir(directory_path)):
    folder_path = os.path.join(directory_path, folder)

    count += 1
    # if count > 10:
    #     break

    # 检查当前路径是否为文件夹
    if os.path.isdir(folder_path):

        # 遍历文件夹下的所有txt文件
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(folder_path, filename)

                # 读取txt文件内容
                with open(file_path, "r", encoding="utf-8") as file:
                    text_content = file.read()

                # 将source和text添加到datas列表中
                datas.append({"source": folder, "text": text_content})

                time.sleep(0.05)

100%|██████████| 288/288 [56:43<00:00, 11.82s/it]


In [51]:
original_datas = datas
# datas = []

n_tokens = []

datas = []

for data in original_datas:
    source = data["source"]
    text = data["text"]
    n_token = len(enc.encode(text))
    n_tokens.append(n_token)
    if n_token > 8000:
        divide_texts = super_divide(text, 6000, 4)
        for divide_text in divide_texts:
            if len(enc.encode(divide_text)) < 1500:
                continue
            datas.append({"source": source, "text": divide_text.strip()})
    else:
        datas.append({"source": source, "text": text.strip()})


In [54]:
def count_quotes(text):

    quote_chars = '''「」"“”'''
    count = 0
    for char in text:
        if char in quote_chars:
            count += 1
    return count

In [55]:
def count_paragraphs(text):
    lines = text.split("\n")
    count = 0
    for line in lines:
        if line.strip() != "":
            count += 1
    return count

In [56]:
from tqdm import tqdm
n_paragraphs = []
n_quotes = []

for data in tqdm(datas):
    n_paragraphs.append(count_paragraphs(data["text"]))
    n_quotes.append(count_quotes(data["text"]))

100%|██████████| 5198/5198 [00:03<00:00, 1601.29it/s]


对于一个字符串line，检查里面的'“'或者'"'或者'”'出现位置。并把所有位置合并为一个pos的list，如果pos的长度为偶数，则把奇数位置替换为「把偶数位置替换为」，请用python为我实现

In [57]:
def change_quotes(text):
    lines = text.split("\n")

    quotes = ['“', '”', '"']

    new_text = ""
    for line in lines:
        if line.strip() == "":
            continue
        positions = []
        for i, char in enumerate(line):
            if char in quotes:
                positions.append(i)
        if len(positions) % 2 == 0:
            for i in range(len(positions)):
                if i % 2 != 0:
                    line = line[:positions[i]] + '」' + line[positions[i]+1:]
                else:
                    line = line[:positions[i]] + '「' + line[positions[i]+1:]
        new_text += line + "\n"
    return new_text


In [58]:
prompt_templates = """Proceed with the paragraph, maintaining the same style of language.
Keep writing the paragraph, using a similar linguistic style.
Continue the paragraph, ensuring the language style remains consistent.
Go on with the paragraph in a similar style of language.
Carry on writing the paragraph, sticking to the current language style.
Extend the paragraph, keeping the language tone and style similar.
Resume the paragraph, mirroring the existing language style.
Continue crafting the paragraph, maintaining the established language style.
Keep extending the paragraph, following the same style of language.
Proceed with extending the paragraph, preserving the linguistic style.
Please compose a paragraph maintaining a uniform style throughout.
Write a paragraph ensuring that the style is consistent from start to finish.
Your assignment is to craft a paragraph with a consistent stylistic approach.
Create a paragraph where the style remains constant throughout.
Construct a paragraph, keeping the writing style uniform in the entire passage.
Develop a paragraph that demonstrates consistency in style.
Focus on writing a paragraph with a steady and unvarying style.
Your challenge is to author a paragraph that maintains a consistent style.
Compose a paragraph, ensuring stylistic consistency throughout.
Produce a paragraph in which the style does not vary from beginning to end.
Your task is to write paragraph in a consistent style
对于下面的Paragraph，使用一致性的风格进行续写
请继续下面的段落，并保持原有的写作风格一致。
在接下来的写作中，延续这一段落的风格，并进行扩展。
针对以下段落，以相同的风格进行补充写作。
续写下面的段落，并确保风格与原文保持一致。
请以与下面段落相同的风格，进行创作性的续写。
保持原段落的写作风格，对其进行扩展和深化。
延续以下段落的风格，进行相应的续写工作。
请在下面的段落后续写，确保风格的连贯性。
继续这一段落的故事，注意保持原有风格的一致性。
针对下文，以一种风格上连贯的方式进行创作性扩写。"""

prompt_templates = prompt_templates.split("\n")

for i in range(len(prompt_templates)):
    prompt_templates[i] = prompt_templates[i].strip()

In [59]:
def segment_paragraph(text):
    raw_lines = text.split("\n")
    lines = []

    all_token_count = 0

    for line in raw_lines:
        line = line.strip()
        if line == "":
            continue
        n_token = len(enc.encode(line))
        all_token_count += n_token

        if n_token > 600:
            segments = super_divide(line)
            lines.extend(segments)
        else:
            lines.append(line)

    seg_token = all_token_count // 3

    if seg_token > 1500:
        seg_token = 1500

    current_seg_token = 0
    current_seg = ""

    segments = []
    for line in lines:
        line = line.strip('"#-。，, \n')
        if line.strip() == "":
            continue
        n_token = len(enc.encode(line))
        if current_seg_token + n_token > seg_token:
            if current_seg_token > 0:
                segments.append(current_seg.strip())
                current_seg = line
                current_seg_token = n_token
        else:
            current_seg += "\n" + line
            current_seg_token += n_token

    if current_seg_token > 0:
        segments.append(current_seg.strip())

    return segments

In [60]:
def find_first_sep( text ):
    first_sep = 9999
    first_sep_second_choice = 9999

    find_first_ket = text[:50].find("」",0)

    if find_first_ket > 0:
        return find_first_ket

    if len(text) <= 50:
        return len(text)
    for i, c in enumerate(text):
        if i > 50:
            break
        if c in ['。','\n']:
            first_sep = min(first_sep, i)
        elif c in ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']:
            first_sep_second_choice = min(first_sep_second_choice, i)

    for i in range(5):
        if len(text) > first_sep and text[first_sep+1] in ['\n', '.', '，', '、', ';', ',', '；',\
                    '：', '！', '？', '(', ')', '”', '“', \
                    '’', '‘', '[', ']', '{', '}', '<', '>', \
                    '/', '''\''', '|', '-', '=', '+', '*', '%', \
                '$', '''#''', '@', '&', '^', '_', '`', '~',\
                    '·', '…','「','」']:
            first_sep += 1

    if first_sep <= 50:
        return first_sep
    elif first_sep_second_choice <= 50:
        return first_sep_second_choice
    else:
        return 50

In [61]:
count = 0

import random

save_datas = []

for i in tqdm(range(len(datas))):

    source = datas[i]["source"]
    # source_pool.add(source)

    text = datas[i]["text"]
    text = change_quotes( text )
    segments = segment_paragraph( text )

    if len(segments) < 3:
        continue

    context = random.choice(prompt_templates)

    context += "\nParagraph:\n###\n" + segments[0] + "\n###\n"



    first_sep = find_first_sep(segments[1])

    context += segments[1][:first_sep+1]

    target = segments[1][first_sep+1:]

    for j in range(2, len(segments)):
        target += "\n\n" + segments[j]

    save_data = {
        "context": context,
        "target": target
    }

    save_datas.append(save_data)


100%|██████████| 5198/5198 [02:45<00:00, 31.41it/s]


In [62]:
print(len(save_datas))

1621


In [67]:
import json

save_name = "/content/erotics.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data in save_datas:
        data['target'] = data['target'].strip()
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')

In [68]:
print(len(save_datas))

1621
